In [1]:
from utils import create_facts_and_examples
import numpy as np
import pandas as pd
import random, os


In [2]:
N = 30
np.random.seed(0) #Code B
# A =  np.bool_(np.random.randint(2, size=N))
# B =  np.bool_(np.random.randint(2, size=N))
A =  np.bool_(np.random.randint(2, size=N))
B =  np.bool_(np.random.randint(2, size=N))
C =  np.bool_(np.random.randint(2, size=N))

customer = ['c'+str(i) for i in range(90)]

Fraud = A & B #& C

df = pd.DataFrame({'orig':np.random.choice(customer, size=N), 'destination':np.random.choice(customer, size=N),\
                   'A':A, 'B':B, "Fraud":Fraud})
df.drop(df[(A==0)&(B==0)].index, inplace=True)

df = pd.DataFrame({'orig':np.random.choice(customer, size=N), 'destination':np.random.choice(customer, size=N),\
                   'A':A, 'B':B, "Fraud":Fraud})
df.drop(df[(A==0)&(B==0)].index, inplace=True)
df['Fraud_chain--orig--destination']=False
df.drop(df[df['orig']==df['destination']].index, inplace=True)

In [3]:
def create_chain(df_):
    df=df_.copy()
    fraudsters = df[df['Fraud']==True]
    index_from_merge = pd.merge(df.reset_index(), fraudsters[['destination']].drop_duplicates(), \
                                how='inner', left_on='orig', right_on='destination')

    # df.loc[index_from_merge['index'],'Fraud']=True
    df.loc[index_from_merge['index'],'Fraud_chain--orig--destination']=True
    df['From--orig']=1
    df['To--destination']=1
    return df

In [4]:
df = create_chain(df)


### creating fact, positive and negative examples

In [5]:
#drop first letter and convert to int
df['orig'] = df['orig'].str[1:].astype('int')
df['destination'] = df['destination'].str[1:].astype('int')

In [6]:
%%script true
create_facts_and_examples(df_= df, target='Fraud', 
                          predicates=['A', 'B', 'From--orig', 'To--destination'], output_dir="examples/chain-fraud-erman")

In [7]:
%%script true
create_facts_and_examples(df_= df, target='Fraud_chain--orig--destination', 
                          predicates=['Fraud', 'From--orig', 'To--destination'], output_dir="examples/transitivity-fraud-rule")

## Create simple transitivity problem

In [8]:
pd.merge(df[['orig','destination']], df[['orig','destination']], \
                            how='inner', left_on=['orig', 'destination'], right_on=['destination','orig'])

,orig_x,destination_x,orig_y,destination_y


In [9]:
df['Fraud--orig--destination']=df['Fraud']
df['Transfer--orig--destination']=1
create_facts_and_examples(df_= df, target='Fraud_chain--orig--destination', 
                          predicates=['Fraud--orig--destination', 'Transfer--orig--destination'], 
                          output_dir="examples/simple-transitivity-fraud-rule", filter_null_columns=False)

Fraud_chain--orig--destination 86 14 flag= True
Fraud_chain--orig--destination 11 67 flag= True
Fraud_chain--orig--destination 32 75 flag= True
Fraud_chain--orig--destination 24 33 flag= True
Fraud_chain--orig--destination 13 85 flag= True
Fraud_chain--orig--destination 58 66 flag= False
Fraud_chain--orig--destination 65 52 flag= False
Fraud_chain--orig--destination 36 67 flag= False
Fraud_chain--orig--destination 43 4 flag= False
Fraud_chain--orig--destination 2 11 flag= False
Fraud_chain--orig--destination 51 86 flag= False
Fraud_chain--orig--destination 80 77 flag= False
Fraud_chain--orig--destination 54 56 flag= False
Fraud_chain--orig--destination 38 24 flag= False
Fraud_chain--orig--destination 19 29 flag= False
Fraud_chain--orig--destination 46 21 flag= False
Fraud_chain--orig--destination 42 25 flag= False
Fraud_chain--orig--destination 56 80 flag= False
Fraud_chain--orig--destination 77 61 flag= False
Fraud_chain--orig--destination 2 32 flag= False
Fraud_chain--orig--destinati

In [10]:
df.reset_index()[['orig','destination','Fraud','Fraud_chain--orig--destination']].iloc[0:9]

,orig,destination,Fraud,Fraud_chain--orig--destination
0,58,66,False,False
1,65,52,False,False
2,36,67,True,False
3,86,14,True,True
4,43,4,True,False
5,11,67,False,True
6,2,11,True,False
7,51,86,True,False
8,80,77,False,False


In [11]:
df[df['Fraud']==True]

,orig,destination,A,B,Fraud,Fraud_chain--orig--destination,From--orig,To--destination,Fraud--orig--destination,Transfer--orig--destination
2,36,67,True,True,True,False,1,1,True,1
4,86,14,True,True,True,True,1,1,True,1
5,43,4,True,True,True,False,1,1,True,1
7,2,11,True,True,True,False,1,1,True,1
8,51,86,True,True,True,False,1,1,True,1
13,38,24,True,True,True,False,1,1,True,1
19,77,61,True,True,True,False,1,1,True,1
21,24,33,True,True,True,True,1,1,True,1
22,2,32,True,True,True,False,1,1,True,1
26,72,13,True,True,True,False,1,1,True,1


In [12]:
df[['orig','destination', 'Fraud_chain--orig--destination']].reset_index()

,index,orig,destination,Fraud_chain--orig--destination
0,0,58,66,False
1,1,65,52,False
2,2,36,67,False
3,4,86,14,True
4,5,43,4,False
5,6,11,67,True
6,7,2,11,False
7,8,51,86,False
8,9,80,77,False
9,10,32,75,True


In [13]:
%%script true
create_facts_and_examples(df_= df, target='Fraud_chain--orig--destination', 
                          predicates=['Fraud--orig--destination'], output_dir="examples/fraud-and-fraud-chain")